1. Введение

In [3]:
from sympy import *
 
x, y = symbols('x, y')
eq1 = Eq(-4*x**3+18*x**2-8*x, 0)
 
sol = solve([eq1], [x])
sol


[(0,), (1/2,), (4,)]

In [4]:
-4**4+6*4**3-4*4**2 + 80

144

In [5]:
x, y, w = symbols('x, y, w')
eq1 = Eq(2*x-w, 0)
eq2 = Eq(4*y-w, 0)
eq3 = Eq(x+y-20, 0)
 
sol = solve([eq1, eq2, eq3], [x,y,w])
13**2+2*7**2


267

2. Градиентный спуск: применение и модификации

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn import linear_model
df = sns.load_dataset('diamonds')

In [7]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [8]:
df = pd.get_dummies(df, drop_first=True)

In [9]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [10]:
X = df.drop(columns="price")
y = df["price"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42,  test_size = 0.33)

In [12]:
param_grid = {"loss": ["squared_error", "epsilon_insensitive"],
              "penalty": ["elasticnet"],
              "alpha": np.logspace(-3, 3, 10),
              "l1_ratio": np.linspace(0, 1, 10),
              "learning_rate": ["constant"],
              "eta0": np.logspace(-4, -1, 4)
              }
grid_search = GridSearchCV(
    estimator=linear_model.SGDRegressor(
        random_state=42,
        # max_iter=1000
    ), 
    param_grid=param_grid, 
    # cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 

y_test_pred = grid_search.predict(X_test)
print(f'MSE на тестовом наборе: {(root_mean_squared_error(y_test, y_test_pred))**2:.3f}')
print(f"Наилучшие значения гиперпараметров: {grid_search.best_params_}")

sgd = linear_model.SGDRegressor(**grid_search.best_params_, random_state = 42)

sgd.fit(X_train, y_train)
sgd.score(X_train, y_train) # r2
ls = sgd.predict(X_test)

round(root_mean_squared_error(y_test, ls), 3)

KeyboardInterrupt: 

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

3. Метод Ньютона

Задание 3.1
1/1 point (graded)
Найдите третий корень полинома , взяв за точку старта 0.7. Введите получившееся значение с точностью до трёх знаков после точки-разделителя.

Попробуйте реализовать алгоритм с использованием Python по аналогии алгоритма градиентного спуска, изученного в предыдущем модуле.

In [10]:
x=0.7
def sent(x):
    return x - (6*x**5 - 5*x**4 -4*x**3 + 3*x**2)/(30*x**4 - 20*x**3-12*x**2+6*x)
while round(sent(x),3) != round(x, 3):
    x = sent(x)
print(x)

0.6286680781673306


In [8]:
def func1(x):
    return 6*x**5-5*x**4-4*x**3+3*x**2
 
def func2(x):
    return 30*x**4-20*x**3-12*x**2+6*x
init_value = 0.7
iter_count = 0
x_curr = init_value
epsilon = 0.000001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
print(iter_count)

0.6296335078534031
0.6286680781673306
0.6286669787778999
0.6286669787764609
4


In [11]:
def func1(x):
    return 3*x**2 - 6*x -45
def func2(x):
    return 6*x - 6

In [12]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

#21.695121951219512
#11.734125501243229
#7.1123493600499685
#5.365000391507974
#5.015260627016227
#5.000029000201801
#5.000000000105126
#5.0

21.695121951219512
11.734125501243229
7.1123493600499685
5.365000391507974
5.015260627016227
5.000029000201801
5.000000000105126
5.000000000000001


In [13]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=func1, fprime=func2, x0=50, tol=0.0001)
# 5

5.0

In [14]:
from scipy.optimize import newton
newton(func=func1, fprime=func2, x0=50, tol=0.0001)

5.0

Задание 3.6
1 point possible (graded)
Дана функция . Найдите решение уравнения  для поиска корня в окрестностях точки . Ответ округлите до трёх знаков после точки-разделителя.
  нет ответа 

In [25]:
x=2.2
f = x**3-72*x -220
fd = 3*x**2 - 72
def sent(x):
    return x - (x**2+9*x-5)/(2*x+9)
while round(sent(x),2) != round(x, 2):
    x = sent(x)
    print(x)
print(x)

0.7343283582089555
0.5291259698087832
0.5249395544696249
0.5249395544696249


In [37]:
x=42
f = x**3-72*x -220
fd = 3*x**2 - 72
def sent(x):
    return x - (8*x**3-2*x**2-450)/(24*x**2-4*x)
while abs(sent(x)-x) > 0.00001:
    x = sent(x)
    print(x)
print(x)

28.038560045532158
18.74430994370089
12.578078092339663
8.533642493466175
5.980028266734071
4.5546096843999795
4.003425702022055
3.9186237380674727
3.916720146800278
3.916720146800278


In [38]:
def func(x):
    return 8*x**3-2*x**2-450
def func1(x):
    return 24*x**2 - 4*x 
def func2(x):
    return 48*x -4
 
init_value = 42
iter_count = 0
x_curr = init_value
epsilon = 0.0001
f = func1(x_curr)
 
while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)
 
print(round(x_curr, 3))
print(round(func(x_curr),3))

21.041749502982107
10.562707090133793
5.323351550447383
2.7040050774153417
1.3949941413301903
0.7418109325525483
0.41784523900811205
0.26096925221473555
0.19169814030401197
0.16955770984744145
0.1667151339969682
0.1666666807529666
0.16666666666666785
0.167
-450.019


4. Квазиньютоновские методы

In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
def func(x):
    return x[0]**2.0 + x[1]**2.0

In [3]:
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

In [4]:
x_0 = [1.0, 1.0]

In [5]:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

In [6]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))
 
#Статус оптимизации Optimization terminated successfully.
#Количество оценок: 3
#Решение: f([0. 0.]) = 0.00000

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [7]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [8]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 -x[0]*x[1]+ x[1]**2.0 +9*x[0]-6*x[1]+20
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] -x[1] + 9, -x[0] +2*x[1] - 6])
 
# определяем начальную точку
x_0 = [-400, -400]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 9
Решение: f([-3.99999972  1.00000028]) = -1.00000


Задание 4.4
1 point possible (graded)
Найдите минимум функции  с помощью квазиньютоновского метода BFGS.

В качестве стартовой точки возьмите .

В качестве ответа введите минимальное значение функции в достигнутой точке.

In [9]:
# определяем нашу функцию
def func(x):
    return x[0]**2 - 3*x + 45
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = [10]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 5
Решение: f([1.5]) = 42.75000


C:\Users\zhigu\AppData\Local\Temp\ipykernel_47804\4129764494.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Решение: f(%s) = %.5f' % (solution, evaluation))


In [10]:
# определяем нашу функцию
def func(x):
    return x[0]**2 - 3*x + 45
 
#  определяем градиент функции
def grad_func(x):
    return np.array([2*x[0] - 3])
 
# определяем начальную точку
x_0 = [10]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([1.5]) = 42.75000


C:\Users\zhigu\AppData\Local\Temp\ipykernel_47804\4291518699.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Решение: f(%s) = %.5f' % (solution, evaluation))


Задание 4.7
2 points possible (graded)
Найдите минимум функции , взяв за стартовую точку .

Какой алгоритм сошелся быстрее?

In [13]:
# определяем нашу функцию
def func(x):
    return x[0]**4 +6*x[1]**2 + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Количество оценок: 40
Решение: f([-9.52718297e-03 -2.32170510e-06]) = 10.00000


In [14]:
# определяем нашу функцию
def func(x):
    return x[0]**4 +6*x[1]**2 + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4*x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 37
Решение: f([1.31617159e-02 6.65344582e-14]) = 10.00000


 6. Практика: линейное программирование

In [23]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [24]:

c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [16]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [17]:
import cvxpy

In [25]:
x = cvxpy.Variable(shape=n, integer = True)

In [26]:
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

In [27]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [28]:
problem.solve()

SolverError: Solver 'SCIPY' failed. Try another solver, or solve with verbose=True for more information.

In [29]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([-0., -0., -0.,  7., -0.,  0.])

In [30]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([1., 0., 0., 1., 0., 1.])

In [31]:
from pulp import *

In [32]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Projects\DS\lessons\.venv\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Задание 6.1
1 point possible (graded)
Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

img

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа.

In [34]:
problem = LpProblem('Распределение запасов', LpMinimize)
x_1 = LpVariable('1-1', lowBound=0 , cat=LpInteger)
x_2 = LpVariable('1-2', lowBound=0 , cat=LpInteger)
x_3 = LpVariable('1-3', lowBound=0 , cat=LpInteger)
y_1 = LpVariable('2-1', lowBound=0 , cat=LpInteger)
y_2 = LpVariable('2-2', lowBound=0 , cat=LpInteger)
y_3 = LpVariable('2-3', lowBound=0 , cat=LpInteger)

#Целевая функция
problem += 2*x_1 + 5*x_2 +3*x_3 + 7*y_1 +7*y_2 +6*y_3
#Ограничения
problem += x_1+x_2+x_3 <=180
problem += y_1+y_2+y_3 <=220
problem += x_1 + y_1 == 110
problem += x_2 + y_2 == 150
problem += x_3 + y_3 == 140
problem.solve()
print("Количество автомобилей модели А: ", x_1.varValue)
print("Количество автомобилей модели В: ", x_2.varValue)
print("Количество автомобилей модели А: ", x_3.varValue)
print("Количество автомобилей модели В: ", y_1.varValue)
print("Количество автомобилей модели А: ", y_2.varValue)
print("Количество автомобилей модели В: ", y_3.varValue)
print("Суммарный доход: ", value(problem.objective))

Количество автомобилей модели А:  110.0
Количество автомобилей модели В:  0.0
Количество автомобилей модели А:  70.0
Количество автомобилей модели В:  0.0
Количество автомобилей модели А:  150.0
Количество автомобилей модели В:  70.0
Суммарный доход:  1900.0


c:\Projects\DS\lessons\.venv\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Задание 6.2
1 point possible (graded)
В прошлом юните мы обсуждали задачу о назначениях исполнителей задач - теперь пришло время решить её.

Напомним суть: необходимо распределить пять задач между пятью исполнителями таким образом, чтобы суммарные затраты на работы были наименьшими.

img

В качестве ответа введите минимальную стоимость работ.

In [46]:
values = np.array([[1000, 12, 10, 19, 8],
                   [12, 1000, 3, 7, 2],
                   [10, 3, 1000, 6, 20],
                   [19, 7, 6, 1000, 4],
                   [8, 2, 20, 4, 1000]])
                   

In [47]:
x = cvxpy.Variable(shape=n, boolean = True)

In [73]:
x = cvxpy.Variable(shape=(5,5), boolean=True)
# constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
sum_0 = cvxpy.sum(x[:, 0]) == 1
sum_1 = cvxpy.sum(x[:, 1]) == 1
sum_2 = cvxpy.sum(x[:, 2]) == 1
sum_3 = cvxpy.sum(x[:, 3]) == 1
sum_4 = cvxpy.sum(x[:, 4]) == 1
sum_00 = cvxpy.sum(x[0, :]) == 1
sum_01 = cvxpy.sum(x[1, :]) == 1
sum_02 = cvxpy.sum(x[2, :]) == 1
sum_03 = cvxpy.sum(x[3, :]) == 1
sum_04 = cvxpy.sum(x[4, :]) == 1

# sum_5 = cvxpy.sum(x[5]) == 1
total_value = cvxpy.sum(cvxpy.multiply(values, x))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[ x_positive, sum_1, sum_0, sum_2, sum_3, sum_4, sum_00, sum_01, sum_02, sum_03, sum_04]
)

print(problem.solve())
x.value

32.0


array([[ 0.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1., -0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0., -0.],
       [-0.,  0.,  0.,  1.,  0.]])

Задание 6.3
1 point possible (graded)
Найдите кратчайший маршрут из точки , который проходит через все другие точки по одному разу и возвращается в .

img

Сформулируйте эту задачу как задачу ЦЛП и решите её. В качестве ответа укажите длину кратчайшего пути.

In [78]:
values = np.array([[1000, 12, 10, 19, 8],
          [12, 1000, 3, 7, 2], 
          [10, 3, 1000, 6, 20],
          [19, 7, 6, 1000, 4],
          [8, 2, 20, 4, 1000]])

In [79]:
x = cvxpy.Variable(shape=(5,5), boolean=True)
# constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
sum_0 = cvxpy.sum(x[:, 0]) == 1
sum_1 = cvxpy.sum(x[:, 1]) == 1
sum_2 = cvxpy.sum(x[:, 2]) == 1
sum_3 = cvxpy.sum(x[:, 3]) == 1
sum_4 = cvxpy.sum(x[:, 4]) == 1
sum_00 = cvxpy.sum(x[0, :]) == 1
sum_01 = cvxpy.sum(x[1, :]) == 1
sum_02 = cvxpy.sum(x[2, :]) == 1
sum_03 = cvxpy.sum(x[3, :]) == 1
sum_04 = cvxpy.sum(x[4, :]) == 1

# sum_5 = cvxpy.sum(x[5]) == 1
total_value = cvxpy.sum(cvxpy.multiply(values, x))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[ x_positive, sum_1, sum_0, sum_2, sum_3, sum_4, sum_00, sum_01, sum_02, sum_03, sum_04]
)

print(problem.solve())
x.value

32.0


array([[ 0.,  0., -0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1., -0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0., -0.],
       [-0.,  0.,  0.,  1.,  0.]])